## Testing parameters for Kimimaro skeletonization. 

In [ ]:
from cloudvolume import Bbox, Vec, CloudVolume, volumecutout, view
import kimimaro
from meshparty import trimesh_io, trimesh_vtk, skeleton_io
import navis
import numpy as np
import sys
from pathlib import Path
from matplotlib import pyplot as plt


from fanc_seg_utils import neuroglancer_utilities
from fanc_seg_utils import skeletonization



Pick a point inside a neuron to analyze, download a region to skeletonize, and download the mesh for comparison. 

In [ ]:
img_mip = np.array([4.3,4.3,45])
seg_mip = np.array([17.2,17.2,45])

vol = CloudVolume('https://storage.googleapis.com/zetta_lee_fly_vnc_001_segmentation/vnc1_full_v3align_2/realigned_v1/seg/full_run_v1',parallel=True,progress=True,mip=seg_mip)

## Select point and get associated seg_id and mesh
pt = np.array([19931, 113919, 2581]) 
seg = neuroglancer_utilities.seg_from_pt(pt)
mesh = vol.mesh.get(seg) 

## Download point, make sure it is good
label = vol.download_point(pt // np.array([4, 4, 1]) ,size=512, mip = seg_mip)
plt.imshow(label[:,:,256])


Method for searching.  Output is in navis neurons for plotting/manipultion

In [ ]:
def param_search(label,seg,img_res,**kwargs):
    teasar_params={
    'scale': 1,
    'const': 200, # physical units
    'pdrf_exponent': 4,
    'pdrf_scale': 100000,
    'soma_detection_threshold': 1100, # physical units
    'soma_acceptance_threshold': 3500, # physical units
    'soma_invalidation_scale': 1.0,
    'soma_invalidation_const': 300, # physical units
    'max_paths': 75, # default None
  }
    for i in teasar_params.keys():
        if i in kwargs.keys():
            teasar_params[i] = kwargs[i]
    
    if 'dust_threshold' in kwargs.keys():
        dust_threshold = kwargs['dust_threshold']
    else:
        dust_threshold = 500
        
        
    skels = kimimaro.skeletonize(
      label,teasar_params=teasar_params,
      object_ids=[seg], # process only the specified labels
      # extra_targets_before=[ (27,33,100), (44,45,46) ], # target points in voxels
      # extra_targets_after=[ (27,33,100), (44,45,46) ], # target points in voxels
      dust_threshold = dust_threshold, # skip connected components with fewer than this many voxels
      anisotropy=(17,17,45), # default True
      fix_branching=True, # default True
      fix_borders=True, # default True
      fill_holes=False, # default False
      fix_avocados=False, # default False
      progress=True, # default False, show progress bar
      parallel=0, # <= 0 all cpu, 1 single process, 2+ multiprocess
      parallel_chunk_size=100, # how many skeletons to process before updating progress bar
    )
    
    for i in kwargs.keys():
        param = i
        val = kwargs[i]
        print(val)
    
    neurons = []    
    for i in skels.keys():
        sk = skels[i]
       # sk.vertices = ( (sk.vertices ) + (np.array(label.bounds.to_list()[0:3]) * multiplier ))
        sk.vertices =  (sk.vertices) + (np.array(label.bounds.to_list()[0:3]) * np.array([4,4,1])) * img_res
        neuron = navis.TreeNeuron(skeletonization.km_to_swc(sk,radius=True,xyz_scaling=1))
        neuron.name = param + ':' + str(val) 
        neuron.nodes.radius = 0
        neuron.nodes[['x','y','z']] = np.array(neuron.nodes[['x','y','z']])
        neurons.append(neuron)


    
    return(neuron)

In [ ]:
skeleton_search = []
for i in range(10):
    skeleton_search.append(param_search(label,seg,img_mip,dust_threshold=(i*100)))


In [ ]:
mesh_n = navis.MeshNeuron(mesh)
colors = plt.cm.plasma(np.linspace(0,1,11))

navis.plot3d([mesh_n,skeleton_search],alpha = .1,linewidth=5,c=colors)
